In [16]:
%load_ext autoreload
%autoreload 2

import os
import itertools 
from os.path import join
from copy import deepcopy
from pdb import set_trace

import dash
from dash import dcc
from dash import html
import dash_daq as daq
import plotly as py
import plotly.graph_objects as go
import plotly.express as px
from plotly import colors
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State, MATCH, ALL
from dash.exceptions import PreventUpdate

import numpy as np
import pandas as pd
import mne
from IPython.display import display, clear_output

import deepbci as dbci
from deepbci.utils import utils as dtils
import vis_utils as vtils

from deepbci.data_utils.data import Groups, run_group_mutators, compress_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
data_cfg_name = 'vis.yaml'
data_cfg_path = dtils.path_to(os.getcwd(), data_cfg_name)
data_cfg = dtils.load_yaml(data_cfg_path)
vtils.clean_config(data_cfg, keep_keys=['groups', 'mutate'])
data_cfg

{'groups': {'_target_': 'deepbci.data_utils.Groups',
  'data_groups': {'dbci': [{'_target_': 'deepbci.data_utils.data_loaders.load_data',
     'load_method': 'load_to_memory',
     'load_method_kwargs': {'subjects': [1, 2, 3, 4, 5, 6, 7, 8, 9],
      'trials': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
      'data_file': 'eeg.csv',
      'true_fs': False,
      'preload_epoch_indexes': {'generate_sync_epochs': None}},
     'data_loader': {'_target_': 'deepbci.data_utils.data_loaders.OAOutLoader'}},
    {'_target_': 'deepbci.data_utils.data_loaders.load_data',
     'load_method': 'load_to_memory',
     'load_method_kwargs': {'subjects': [1, 2, 3, 4, 5, 6, 7, 8, 9],
      'trials': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
      'data_file': 'eeg.csv',
      'true_fs': False,
      'preload_epoch_indexes': {'generate_sync_epochs': None}},
     'data_loader': {'_target_': 'deepbci.data_utils.data_loaders.OAObsLoader'}},
    {'_target_': 'deepbci.data_utils.data_loaders.load_data',
     'load_method': 'load_

In [18]:
mne.set_log_level('ERROR')
grps = vtils.instantiate_and_mutate(data_cfg)

In [19]:
dropped_df = vtils.get_drop_logs(grps)
display(dropped_df)

percent  dropped  total
group dataset subject trial                          
dbci  BGSInt  1       NaN    0.008721        9   1032
              10      NaN    0.000000        0    522
              11      NaN    0.005780        3    519
              2       NaN    0.016330       17   1041
              3       NaN    0.009551       10   1047
              4       NaN    0.008696        9   1035
              5       NaN    0.042533       45   1058
              6       NaN    0.009443       10   1059
              7       NaN    0.012464       13   1043
              8       NaN    0.000000        0   1042
              9       NaN    0.008662        9   1039
      BGSObs  1       NaN    0.006567        7   1066
              2       NaN    0.018701       19   1016
              3       NaN    0.003868        4   1034
              4       NaN    0.001936        2   1033
              5       NaN    0.004854        5   1030
              6       NaN    0.006585        7   1063
              7       NaN    0.007707        8   1038
              8       NaN    0.002860        3   1049
              9       NaN    0.006686        7   1047
      OAObs   1       NaN    0.007937        1    126
              2       NaN    0.016393        2    122
              3       NaN    0.000000        0    108
              4       NaN    0.000000        0    117
              5       NaN    0.000000        0    105
              6       NaN    0.000000        0    115
              7       NaN    0.018519        2    108
              8       NaN    0.017699        2    113
              9       NaN    0.000000        0     99
      OAOut   1       NaN    0.000000        0     89
              2       NaN    0.070423        5     71
              3       NaN    0.069307        7    101
              4       NaN    0.013514        2    148
              5       NaN    0.115942        8     69
              6       NaN    0.027451        7    255
              7       NaN    0.006711        1    149
              8       NaN    0.000000        0     70
              9       NaN    0.021739        2     92

In [20]:
print(grps)

group  dataset  subject  trial
dbci   BGSInt   1        NaN      DataGroup(BaseEpochs, ndarray)
                10       NaN      DataGroup(BaseEpochs, ndarray)
                11       NaN      DataGroup(BaseEpochs, ndarray)
                2        NaN      DataGroup(BaseEpochs, ndarray)
                3        NaN      DataGroup(BaseEpochs, ndarray)
                4        NaN      DataGroup(BaseEpochs, ndarray)
                5        NaN      DataGroup(BaseEpochs, ndarray)
                6        NaN      DataGroup(BaseEpochs, ndarray)
                7        NaN      DataGroup(BaseEpochs, ndarray)
                8        NaN      DataGroup(BaseEpochs, ndarray)
                9        NaN      DataGroup(BaseEpochs, ndarray)
       BGSObs   1        NaN      DataGroup(BaseEpochs, ndarray)
                2        NaN      DataGroup(BaseEpochs, ndarray)
                3        NaN      DataGroup(BaseEpochs, ndarray)
                4        NaN      DataGroup(BaseEpochs, nda

In [21]:
# Example of how to index Groups.data_map
grps['dbci', ['OAOut', 'BGSInt']]

array([DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray)],
      dtype=object)

In [22]:
# Example of how to use Groups.get_series()
grps.get_series(['dbci', ['OAOut', 'BGSInt']])

group  dataset  subject  trial
dbci   OAOut    1        NaN      DataGroup(BaseEpochs, ndarray)
                2        NaN      DataGroup(BaseEpochs, ndarray)
                3        NaN      DataGroup(BaseEpochs, ndarray)
                4        NaN      DataGroup(BaseEpochs, ndarray)
                5        NaN      DataGroup(BaseEpochs, ndarray)
                6        NaN      DataGroup(BaseEpochs, ndarray)
                7        NaN      DataGroup(BaseEpochs, ndarray)
                8        NaN      DataGroup(BaseEpochs, ndarray)
                9        NaN      DataGroup(BaseEpochs, ndarray)
       BGSInt   1        NaN      DataGroup(BaseEpochs, ndarray)
                10       NaN      DataGroup(BaseEpochs, ndarray)
                11       NaN      DataGroup(BaseEpochs, ndarray)
                2        NaN      DataGroup(BaseEpochs, ndarray)
                3        NaN      DataGroup(BaseEpochs, ndarray)
                4        NaN      DataGroup(BaseEpochs, nda

In [23]:
# Example of how to use Groups.get_series()
grps.get_array(['dbci', ['OAOut', 'BGSInt']])

array([DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray)],
      dtype=object)

In [24]:
grps.get_array(['dbci', ['OAOut', 'BGSInt']])

array([DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray),
       DataGroup(BaseEpochs, ndarray), DataGroup(BaseEpochs, ndarray)],
      dtype=object)

# JupyterDash Visualization

## Manual

In [25]:
import plot as plts

pa = plts.PlotAverages(grps)

test_plots = True

epochs = {'dbci-BGSInt-1-1': 40,
        'dbci-BGSInt-1-2': 40,
        'dbci-OAOut-1-1': 40}

# filter_kws = {
#     'low_pass': 30,
#     'high_pass': .1
# }

In [26]:
def test_plot_subject_average(pa):
    kwargs = dict(
        group = ['dbci'],
        dataset = ['BGSInt', 'BGSObs'],
        subject = ['1'],
        channel = ['F3', 'F4', 'Fz', 'Cz'],
        event = ['1'],
        epochs=epochs,
    )
    return pa.plot_subject_average(**kwargs)

if test_plots:
    plot_df = test_plot_subject_average(pa)
    fig = plot_df
    fig.show()

<class 'str'>
test {'dbci-BGSInt-1-1': 40, 'dbci-BGSInt-1-2': 40, 'dbci-OAOut-1-1': 40} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 40
Combo: ('dbci', 'BGSInt', '1', '1', 'F3')
	Epoch count: 40
Combo: ('dbci', 'BGSInt', '1', '1', 'F4')
	Epoch count: 40
Combo: ('dbci', 'BGSInt', '1', '1', 'Fz')
	Epoch count: 40
Combo: ('dbci', 'BGSObs', '1', '1', 'Cz')
	Epoch count: 211
Combo: ('dbci', 'BGSObs', '1', '1', 'F3')
	Epoch count: 211
Combo: ('dbci', 'BGSObs', '1', '1', 'F4')
	Epoch count: 211
Combo: ('dbci', 'BGSObs', '1', '1', 'Fz')
	Epoch count: 211


In [27]:
def test_plot_grand_average(pa):
    kwargs = dict(
        group = ['dbci'],
        dataset = ['BGSInt', 'BGSObs', 'OAOut', 'OAObs'],
        subject = ['1','2', '3', '4', '5', '6', '7', '8', '9'],
        channel = ['Cz'],
        event = ['1'],
#         epochs = epochs,
#         mne_filter_kws=filter_kws,
    )

    return pa.plot_grand_average(**kwargs)

if test_plots:
    fig = test_plot_grand_average(pa)
    fig.show()

Combo: ('dbci', 'BGSInt', '1', 'Cz')
	Epoch count: 1863
Combo: ('dbci', 'BGSObs', '1', 'Cz')
	Epoch count: 1874
Combo: ('dbci', 'OAObs', '1', 'Cz')
	Epoch count: 1006
Combo: ('dbci', 'OAOut', '1', 'Cz')
	Epoch count: 1012


In [28]:
def test_plot_subject_grand_average(pa):
    kwargs = dict(
        group = ['dbci'],
        dataset = ['BGSInt'],
        subject = ['1','2','3','4','5','6','7','8','9'],
        channel = ['Cz'],
        event = ['1'],
    )

    return pa.plot_subject_grand_average(**kwargs)

if test_plots:
    fig, plot_df = test_plot_subject_grand_average(pa)
    fig.show()

[['dbci'], ['BGSInt'], ['1', '2', '3', '4', '5', '6', '7', '8', '9'], ['Cz'], ['1']]
Dataset: BGSInt Event: 1
	Subject: 1 Count: 186
	Subject: 2 Count: 214
	Subject: 3 Count: 225
	Subject: 4 Count: 191
	Subject: 5 Count: 213
	Subject: 6 Count: 223
	Subject: 7 Count: 211
	Subject: 8 Count: 203
	Subject: 9 Count: 197
	Total: 1863


## Dashboard

In [29]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='group-dropdown',
                options=[dict(label=g, value=g) for g in pa.grps.get_level_values(level='group').unique()],
                placeholder='Groups',
            ),
            dcc.Dropdown(
                id='dataset-dropdown',
                options=[],
                placeholder='Datasets',
                multi=True
            ),
            dcc.Dropdown(
                id='subject-dropdown',
                options=[],
                placeholder='Subjects',
                multi=True
            ),
            dcc.Dropdown(
                id='event-id-dropdown',
                options=[],
                placeholder='Event IDs',
                multi=True
            ),
            dcc.Dropdown(
                id='channel-dropdown',
                options=[],
                placeholder='Channels',
                multi=True
            ),
            
        ], style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),

        html.Div([
            html.Div([
                daq.BooleanSwitch(
                    id='filter-switch',
                    on=False,
                    vertical=False,
                    style= {'width':'20%', 'display': 'inline-block'}
                ),
                dcc.Input(
                    id='high-pass-input', 
                    type='number',
                    value=None,
                    disabled=True,
                    debounce=True,
                    placeholder='High-pass filter bound',
                    style= {'width':'40%', 'display': 'inline-block'}
                ),
                dcc.Input(
                    id='low-pass-input', 
                    type='number',
                    value=None,
                    disabled=True,
                    debounce=True,
                    placeholder='Low-pass filter bound',
                    style= {'width':'40%', 'display': 'inline-block'}
                ),
            ],style={'display': 'flex', 'padding': '10px 5px'}),
            html.Div([
                html.Div([
                    dcc.Dropdown(
                        id='plot-dropdown',
                        options=[
                            {'label' : 'SA', 'value': 'SA'},
                            {'label' : 'GA', 'value': 'GA'},
                            {'label' : 'SA+GA', 'value': 'SA+GA'},
                        ],
                        value='SA',
                        placeholder='Plot Type'
                    )
                ], style={'width': '45%', 'float': 'left', 'display': 'inline-block', 'padding': '5px 25px'}),
            ]),
        ]),
        html.Div(id='epoch-sliders', children=[]),
    ], style={
        'width': '39%', 
        'float': 'left', 
        'display': 'inline-block',
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
        }
    ),
    
    html.Div([
        dcc.Graph(
            id='graph',
            figure={},
            style={
                'width': '79%', 
                'height': '79%', 
            },
        ), 
    ], style={
            'width': '59%', 
            'float': 'right', 
            'display': 'inline-block',
            'text-align': 'center',
            }
    )

])

def get_epoch_keys(data, event):
    values = data.index.droplevel(level='trial')
    key_tuple =  list(values)[0] + (event,)
    key = '-'.join(map(str, key_tuple))
    
    return key, key_tuple

@app.callback(
    Output('dataset-dropdown', 'options'),
    [Input('group-dropdown', 'value')])
def set_dataset_options(group):
    if not group:
        raise PreventUpdate
        
    datasets = list(pa.grps.get_level_values(select=[group], level='dataset').unique())
    
    return [dict(label=d, value=d) for d in datasets]

@app.callback(
    Output('subject-dropdown', 'options'),
    [Input('dataset-dropdown', 'value'),
     Input('group-dropdown', 'value')])
def set_subject_options(dataset, group):
    subjects = list(pa.grps.get_level_values(select=[group, dataset], level='subject').unique())

    return [dict(label=s, value=s) for s in subjects]

@app.callback(
    Output('event-id-dropdown', 'options'),
    [Input('group-dropdown', 'value'),
     Input('dataset-dropdown', 'value'),
     Input('subject-dropdown', 'value')])
def set_event_ids_options(group, dataset, subject):
    data = pa.grps[group, dataset, subject]
    data = data.ravel()
    
    # If no options are found return an empty list
    if len(data) == 0:
        return []
    
    unique_event_ids = np.unique(data[0].data.events[:, -1])
    for d in data[1:]:
        event_id = np.unique(d.data.events[:, -1])
        unique_event_ids = np.unique(np.concatenate([unique_event_ids, event_id]))
        
    return [dict(label=e, value=e) for e in unique_event_ids]

@app.callback(
    Output('channel-dropdown', 'options'),
    [Input('group-dropdown', 'value'),
     Input('dataset-dropdown', 'value'),
     Input('subject-dropdown', 'value')])
def set_channels_options(group, dataset, subject):
    data = pa.grps[group, dataset, subject]
    data = data.ravel()
    
    # If no options are found return an empty list
    if len(data) == 0:
        return []
    
    unique_channels = np.unique(data[0].data.ch_names)
    for d in data[1:]:
        chs = np.unique(d.data.ch_names)
        max_ch = np.unique(np.concatenate([unique_channels, chs]))
        
    return [dict(label=c, value=c) for c in unique_channels]

@app.callback(
    [Output('high-pass-input', 'disabled'),
     Output('low-pass-input', 'disabled'),
     Output('high-pass-input', 'value'),
     Output('low-pass-input', 'value')],
    [Input('filter-switch', 'on')],
    [State('high-pass-input', 'value'),
     State('low-pass-input', 'value')])
def enable_filtering(enable_filter, high_pass, low_pass):
    if enable_filter:
        return False,  False, high_pass, low_pass
    else:
        return True, True, None, None
    
@app.callback(
    [Output('epoch-sliders', 'children')],
    [Input('group-dropdown', 'value'),
     Input('dataset-dropdown', 'value'),
     Input('subject-dropdown', 'value'),
     Input('event-id-dropdown', 'value')],
    [State({'type': 'dynamic-sliders', 'index': ALL}, 'value'),
     State({'type': 'dynamic-sliders', 'index': ALL}, 'id'),
     State({'type': 'dynamic-sliders', 'index': ALL}, 'max')])   
def set_epoch_count(group, dataset, subject, event, epoch_value, epoch_id, epoch_max):
    children = []

    if None in (group, dataset, subject, event):
        raise PreventUpdate
    
    data = pa.grps.data_map.loc[group, dataset, subject]
    
    if len(data.ravel()) == 0:
        raise PreventUpdate
    
    prev_epochs = {i['index']: [v, m] for i,v,m in zip(epoch_id, epoch_value, epoch_max)}

    for _, d in data.groupby(['group', 'dataset', 'subject']):
        for e in event:
            key, _ = get_epoch_keys(d, e)
            if key in prev_epochs:
                e_max = prev_epochs[key][1]
                e_value =  prev_epochs[key][0]
            else:
                e_max = len(d[0].data.events[d[0].data.events == e])
                e_value = e_max
            new_element = html.Div([
               html.Label(key, style={'width':'10%', 'display': 'inline-block',}),
               dcc.Slider(
                    id={'type': 'dynamic-sliders', 'index': key},
                    step=1,
                    min=0,
                    max=e_max,
                    value=e_value,
                    updatemode='mouseup',
                    tooltip=dict(always_visible=True)
               )
            ], style= {'width':'90%', 'display': 'inline-block'})
            children.append(new_element)

    return [children]

@app.callback(
    Output('graph', 'figure'),
    [Input('group-dropdown', 'value'),
     Input('dataset-dropdown', 'value'),
     Input('subject-dropdown', 'value'),
     Input('event-id-dropdown', 'value'),
     Input('channel-dropdown', 'value'),
     Input('high-pass-input', 'value'),
     Input('low-pass-input', 'value'),
     Input('plot-dropdown', 'value'),
     Input({'type': 'dynamic-sliders', 'index': ALL}, 'value')],
    [State({'type': 'dynamic-sliders', 'index': ALL}, 'id')])
def display_graph(
    group, dataset, subject, event, channel, 
    high_pass, low_pass, plot_type,
    epoch_values, epoch_ids
):
    plot_args = (group, dataset, subject, event, channel)
    # Bandaid fix for catching exceptions as dropdown menu options are filled out.
    # Ideally, all options would be completed before plotting, this is an easier
    # solution for now.
    if None in plot_args:
        raise PreventUpdate # return px.line()
    elif any([len(arg) == 0 for arg in plot_args]):
        raise PreventUpdate # return px.line()
    
    mne_filter_kws = dict(low_pass=low_pass, high_pass=high_pass)

    # Convert dyanmic sliders into dict of values
    # parse slider id to get indexes
    epochs = {i['index']: v for i, v in zip(epoch_ids, epoch_values)}
    print(epochs)
    if plot_type == 'GA':
        fig = pa.plot_grand_average(group=group,
                                dataset=dataset,
                                subject=subject,
                                event=event,
                                channel=channel,
                                mne_filter_kws=mne_filter_kws,
                                epochs=epochs)
    elif plot_type == 'SA':
        fig = pa.plot_subject_average(group=group,
                                  dataset=dataset,
                                  subject=subject,
                                  event=event,
                                  channel=channel,
                                  mne_filter_kws=mne_filter_kws,
                                  epochs=epochs)
    elif plot_type == 'SA+GA':
        fig, _ = pa.plot_subject_grand_average(group=group,
                                            dataset=dataset,
                                            subject=subject,
                                            event=event,
                                            channel=channel,
                                            mne_filter_kws=mne_filter_kws,
                                            epochs=epochs)
    return fig

In [30]:
app.run_server(mode="external", host='0.0.0.0', port=8866)
# app.run_server(mode='inline', host='0.0.0.0', port=8866)

Dash app running on http://0.0.0.0:8866/


/home/dev/mnt/deepbci/data_utils/data.py:93: FutureWarning:

The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`

/home/dev/mnt/deepbci/data_utils/data.py:291: FutureWarning:

The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`

/home/dev/mnt/deepbci/data_utils/data.py:93: FutureWarning:

The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence'

{'dbci-BGSInt-1-1': 186}
<class 'str'>
test {'dbci-BGSInt-1-1': 186} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 186
{'dbci-BGSInt-1-1': 71}
<class 'str'>
test {'dbci-BGSInt-1-1': 71} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 71
{'dbci-BGSInt-1-1': 118}
<class 'str'>
test {'dbci-BGSInt-1-1': 118} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 118
{'dbci-BGSInt-1-1': 46}
<class 'str'>
test {'dbci-BGSInt-1-1': 46} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 46
{'dbci-BGSInt-1-1': 49}
<class 'str'>
test {'dbci-BGSInt-1-1': 49} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 49
{'dbci-BGSInt-1-1': 47}
<class 'str'>
test {'dbci-BGSInt-1-1': 47} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 47
{'dbci-BGSInt-1-1': 56}
<class 'str'>
test {'dbci-BGSInt-1-1': 56} 1 object
Combo: ('dbci', 'BGSInt', '1', '1', 'Cz')
	Epoch count: 56
{'dbci-BGSInt-1-1': 157}
<class 'str'>
test {'dbc